<a href="https://colab.research.google.com/github/CarlosLongarela/Whisper-transcribir-traducir/blob/main/Whisper_transcribir_traducir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transcripción y traducción WHISPER V 0.1
### Cuaderno de colab programado por Carlos Longarela
### https://tabernawp.com/

1. Este cuaderno permite hacer una transcripción o una traducción de un 
audio o video en la mayoria de formatos populares en cualquier idioma o traducirlo al inglés (o ambas tareas). El resultado son archivos de texto (txt, srt y vtt) planos y con los tiempos para usar como subtitulos. Los formatos soportados son los siguientes https://en.wikipedia.org/wiki/FFmpeg#Supported_codecs_and_formats.
2. Los archivos de audio y los archivos de texto generados quedarán almacenados en tu google drive en una carpeta llamada whisper, que se borrará y recreará con cada nueva ejecución.
3. Este cuaderno NO codifica video ni audio, solo genera los archivos de texto preparados para susbtítulos.

# Instrucciones de uso:

1. El entorno de ejecución deberá tener aceleración por hardware de GPU o TPU.
1. Se deben ejecutar las tres tareas secuencialmente.
1. Una vez ejecutadas las tres tareas, en consecuentes ejecuciones solo debemos ejecutar las tareas 2 y 3.
1. Con la primera tarea instalamos Whisper y las librerías necesarias, además de dar permiso a nuestro Drive para poder guardar los archivos.
1. Con la tarea 2 seleccionamos el archivo de audio a utilizar y los parámetros a utilizar en la tarea (ver los parámetros de la tarea 2).
1. Con la tarea 3 realizamos la transcripción/traducción y copia de archivos a su destino definitivo en nuestro Google Drive.


---




In [1]:
#@title 1) Instalamos las dependencias y librerías y montamos el Google Drive.

# Install Whisper
!pip install git+https://github.com/openai/whisper.git > /dev/null 2>&1
!pip install jiwer whisper > /dev/null 2>&1

# Mount Google Drive.
from google.colab import drive
drive.mount('/content/drive')

# Declare this step as done.
step1 = True

Mounted at /content/drive


### 1. En primer lugar deberemos seleccionar los tres siguientes parámetros (o dejar las opciones por defecto).

**task:** L tarea que va a realizarse, traducir, transcribir o ambas.

**model:** Modelo de whisper usado para hacer la tarea, tiny y small son los menos complejos y más rápidos, large es el más complejo y lento, con el que se lograra mejores resultados en general.

**langauge:**  Idioma del audio a transcribir/traducir, que puede ser Español (*Spanish*), Inglés (*Inglés*) o Autodetectar (*Autodetect*).

### 2. A continuación debermos ejecutar la tarea, lo que nos permitirá elegir el archivo de audio para transcribir/convertir en el siguinete paso.

In [2]:
#@title 2) Seleccionamos opciones y subimos el archivo de audio
import os
from google.colab import files
import shutil

folder_name = '/content/drive/MyDrive/whisper'
folder_name_en = '/content/drive/MyDrive/whisper/en'

# If folder exists, we delete it.
if os.path.exists( folder_name ):
  shutil.rmtree( folder_name )

# Create folders.
os.mkdir( folder_name )
os.mkdir( folder_name_en )

task = "Transcription and Translation" #@param ["Transcription and Translation", "Translation", "Transcripcion"]
model = "large" #@param ["large", "medium", "tiny", "tiny.en", "base.en", "base", "small.en", "small", "medium.en", "large-v1", "large-v2"]
language = "Spanish" #@param ["Autodetect", "Spanish", "English"]

if language == 'Autodetect':
  language_param = ''
else:
  language_param = '--language ' + language

uploaded_files = files.upload()
for file in uploaded_files.keys():
  s = file
  file2 = s.replace( ' ', '' )
  s = file2 
  file2 = s.replace( ')', '' )
  s = file2 
  file2 = s.replace( '(', '' )
  destination = os.path.join( folder_name, file2 )
  print( f'Moviendo {file} a {file2}' )
  shutil.copy( file, destination )
  extension = os.path.splitext( file2 )[1]
  name = os.path.splitext( file2 )[0]
  print( f'{name}{extension}' )
if extension != '.mp3':
  !ffmpeg -i {folder_name}/{name}{extension} {folder_name}/{name}.mp3 > /dev/null 2>&1
  destination_new = folder_name + '/' + name + '.mp3'
  !rm {destination} > /dev/null 2>&1
else:
    destination_new = destination


Saving curso-vid-a1004.mp3 to curso-vid-a1004.mp3
Moviendo curso-vid-a1004.mp3 a curso-vid-a1004.mp3
curso-vid-a1004.mp3


### 3. Ejecutamos la tarea encargada de realizar la transcripción/traducción.

In [ ]:
#@title 3) Ejecutar Whisper con los parámetros seleccionados

def cl_transcript():
  print( 'Se va a realizar una transcripción', "\n" )
  print( '-----------------------------------------', "\n")
  !whisper {destination_new} --task transcribe {language_param} --model {model}
  !mv /content/*.srt {folder_name}
  !mv /content/*.txt {folder_name}
  !mv /content/*.vtt {folder_name}
  print( "\n\n" )
  print( '-----------------------------------------', "\n")
  print( 'los archivos de la transcripción fueron copiados en ',  folder_name )
  print( "\n\n" )

def cl_translate():
  print( 'Se va a realizar una traducción', "\n" )
  print( '-----------------------------------------', "\n")
  !whisper {destination_new} --task translate {language_param} --model {model}
  !mv /content/*.srt {folder_name_en}
  !mv /content/*.txt {folder_name_en}
  !mv /content/*.vtt {folder_name_en}
  !cd {folder_name_en}; for file in *; do mv "$file" "en-$file"; done
  print( "\n\n" )
  print( '-----------------------------------------', "\n")
  print( 'los archivos de la traducción fueron copiados en ',  folder_name )
  print( "\n\n" )

if task == "Transcripcion":
  cl_transcript()
elif task == "Translation":
  cl_translate()
else: 
  cl_transcript()
  cl_translate()


Se va a realizar una transcripción 

----------------------------------------- 

100%|█████████████████████████████████████| 2.87G/2.87G [01:10<00:00, 43.8MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Traceback (most recent call last):
  File "/usr/local/bin/whisper", line 8, in <module>
    sys.exit(cli())
  File "/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py", line 448, in cli
    result = transcribe(model, audio_path, temperature=temperature, **args)
  File "/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py", line 229, in transcribe
    result: DecodingResult = decode_with_fallback(mel_segment)
  File "/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py", line 164, in decode_with_fallback
    decode_result = model.decode(segment, options)
  File "/usr/local/lib/python3.10/dist-packages/torc